In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px
import warnings
from sklearn.preprocessing import MinMaxScaler
from keras import Sequential
from keras.src.layers import Dense
from keras.src.layers import LSTM, Dropout
from keras.src.optimizers import Adam
from keras.src.callbacks import ModelCheckpoint, EarlyStopping
import math
from sklearn.metrics import mean_squared_error
import datetime
import mplfinance as mpf
import math 
import os
import plotly.graph_objects as go
df = pd.read_csv('/Users/leo/Desktop/Invest_hub/data/historical_data/VKCO_1h.csv')

In [4]:
def str_to_datetime(s):
    date_part, time_part = s.split(' ')[0], s.split(' ')[1].split('+')[0]
    year, month, day = map(int, date_part.split('-'))
    hour, _, _ = map(int, time_part.split(':'))
    return datetime.datetime(year=year, month=month, day=day, hour=hour, minute=0, second=0)

df['Time'] = df['Time'].apply(str_to_datetime)

def specific_data(start, end):
    data = df
    filtered_data = data[(data['Time'] >= start) & (data['Time'] <= end)]
    return filtered_data

start_time = df['Time'].iloc[-1350]
end_time = df['Time'].iloc[-15]
spec_df = specific_data(start_time, end_time)

In [5]:
new_df = spec_df['Close'].values
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(new_df.reshape(-1, 1))
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]
n_past = 100
X_train, y_train = [], []
for i in range(n_past, len(train_data)):
    X_train.append(train_data[i - n_past:i, 0])
    y_train.append(train_data[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

X_test, y_test = [], []
for i in range(n_past, len(test_data)):
    X_test.append(test_data[i - n_past:i, 0])
    y_test.append(test_data[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)

# Изменение формы данных для LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Построение и обучение модели LSTM
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
checkpoints = ModelCheckpoint(filepath='my_weights.keras', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, verbose=1, callbacks=[checkpoints, early_stopping])


Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 201ms/step - loss: 0.0559 - val_loss: 0.0100
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 189ms/step - loss: 0.0077 - val_loss: 0.0076
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 191ms/step - loss: 0.0042 - val_loss: 0.0079
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 189ms/step - loss: 0.0034 - val_loss: 0.0079
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 190ms/step - loss: 0.0038 - val_loss: 0.0066
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 189ms/step - loss: 0.0031 - val_loss: 0.0064
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 189ms/step - loss: 0.0027 - val_loss: 0.0059
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 191ms/step - loss: 0.0030 - val_loss: 0.0058
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 192ms/step - loss: 0.0029 - val_loss: 0.0054
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 192ms/step - loss: 0.0028 - val_loss: 0.0053
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 193ms/step - loss: 0.0031 - val_loss: 0.0055
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3

In [7]:
last_sequence = X_test[-1].reshape(1, n_past, 1)
predictions_next_24_hours = []
for _ in range(24):
    next_hour_prediction = model.predict(last_sequence)
    predictions_next_24_hours.append(next_hour_prediction[0, 0])
    last_sequence = np.roll(last_sequence, -1, axis=1)
    last_sequence[0, -1, 0] = next_hour_prediction

predictions_next_24_hours = scaler.inverse_transform(np.array(predictions_next_24_hours).reshape(-1, 1))

real_data_next_24_hours = specific_data(df['Time'].iloc[-15], df['Time'].iloc[-1])
real_dates = pd.to_datetime(real_data_next_24_hours['Time'])

predicted_dates = real_dates

fig = go.Figure()
fig.add_trace(go.Scatter(x=real_dates, y=real_data_next_24_hours['Close'], mode='lines', name='Actual Prices (Next 24 Hours)', line=dict(color='black')))
fig.add_trace(go.Scatter(x=predicted_dates, y=predictions_next_24_hours.flatten(), mode='lines+markers', name='Predicted Prices (Next 24 Hours)', line=dict(color='blue')))
fig.update_layout(title='Actual vs Predicted Stock Prices', xaxis_title='Time', yaxis_title='Price', legend=dict(x=0, y=1), template='plotly_white')
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

